In [0]:
import tensorflow as tf
import pandas as pd

In [0]:
# Code to read csv file into Colaboratory:

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
trainlink = "https://drive.google.com/open?id=1U4Yrb5z2ciLXr1YE2jGkMnV1losEYktG"
testlink = "https://drive.google.com/open?id=11axtXwQjC3ig6TPiq101eWv76bw9qm-i"

In [0]:
fluff, id = trainlink.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('fashion-mnist_train.csv')  
train = pd.read_csv('fashion-mnist_train.csv')


fluff, id = testlink.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('fashion-mnist_test.csv')  
test = pd.read_csv('fashion-mnist_test.csv')

# Dataset is now stored in a Pandas Dataframe

In [0]:
trainY = train["label"]
del train["label"]

In [0]:
def onehotencoding(y,num_classes):
  y_true = []
  for i in y:
    temp_list = [0]*num_classes
    temp_list[i] = 1
    y_true.append(temp_list)
  return y_true

In [0]:
y_true = onehotencoding(trainY,10)
x = train

In [25]:
len(y_true)

60000

In [0]:
img_size = 784
img_size_flat = 784
img_shape = (28,28)
num_classes = 10
num_channels = 1

In [0]:
# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 128             # Number of neurons in fully-connected layer.

In [0]:
def new_weights(shape):
  return tf.Variable(tf.truncated_normal(shape, stddev = 0.5))

def new_bias(length):
  return tf.Variable(tf.constant(0.05, shape=[length]))

In [0]:
def new_convolutional_layer(input,
                           number_of_input_channels,
                           filter_size,
                           num_filters,
                           use_pooling=True):
  shape = [filter_size,filter_size,number_of_input_channels,num_filters]
  
  weights = new_weights(shape=shape)
  biases = new_bias(length=num_filters)
  
  layer = tf.nn.conv2d(input=input,
                       filter=weights,
                       strides=[1,1,1,1],
                       padding='SAME') + biases
  
  if use_pooling:
    layer = tf.nn.max_pool(value=layer,
                           ksize=[1, 2, 2, 1],
                           strides=[1, 2, 2, 1],
                           padding='SAME')
    
  layer = tf.nn.relu(layer)
  
  return layer, weights

In [0]:
def flatten_layer(layer):
  layer_shape = layer.get_shape()
  
  num_features = layer_shape[1:4].num_elements()
  
  layer_flat = tf.reshape(layer, [-1,num_features])
  
  return layer_flat,num_features

In [0]:
def new_fc_layer(input,num_inputs,num_outputs,use_relu):
  weights = new_weights(shape=[num_inputs,num_outputs])
  bias = new_bias(length=num_outputs)
  layer = (tf.matmul(input,weights) + bias)
  if use_relu:
    layer = tf.nn.relu(layer)
  return layer

In [0]:
x = tf.placeholder(tf.float32, shape=[None,img_size_flat])
x_image = tf.reshape(x, [-1,img_size,img_size,num_channels])

y_true = tf.placeholder(tf.float32, shape=[None,num_classes])
y_true_cls = tf.argmax(y_true, axis=1)

In [0]:
layer_conv1, weights_conv1 = new_convolutional_layer(input=x_image,
                                                     number_of_input_channels=num_channels,
                                                     filter_size=filter_size1,
                                                     num_filters=num_filters1,
                                                     use_pooling=True)

In [62]:
layer_conv1

<tf.Tensor 'Relu:0' shape=(?, 392, 392, 16) dtype=float32>

In [0]:
layer_conv2, weights_conv2 = new_convolutional_layer(input=layer_conv1,
                                                     number_of_input_channels=num_filters1,
                                                     filter_size=filter_size2,
                                                     num_filters=num_filters2,
                                                     use_pooling=True)

In [64]:
layer_conv2

<tf.Tensor 'Relu_1:0' shape=(?, 196, 196, 36) dtype=float32>

In [0]:
layer_flat,num_features = flatten_layer(layer_conv2)

In [66]:
layer_flat

<tf.Tensor 'Reshape_4:0' shape=(?, 1382976) dtype=float32>

In [67]:
num_features

1382976

In [0]:
layer_fc1 = new_fc_layer(input=layer_flat,
                     num_inputs=num_features,
                     num_outputs=fc_size,
                     use_relu=True)

In [74]:
layer_fc1

<tf.Tensor 'Relu_3:0' shape=(?, 128) dtype=float32>

In [0]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                     num_inputs=fc_size,
                     num_outputs=num_classes,
                     use_relu=False)

In [76]:
layer_fc2

<tf.Tensor 'add_6:0' shape=(?, 10) dtype=float32>

In [0]:
y_pred = tf.nn.softmax(layer_fc2)


In [0]:
y_pred_cls = tf.argmax(y_pred, axis=1)

In [0]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2,
                                                       labels=y_true)

In [0]:
cost = tf.reduce_mean(cross_entropy)

In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [0]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)


In [0]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
session = tf.Session()

In [0]:
session.run(tf.global_variables_initializer())

In [0]:
train_batch_size = 60

In [0]:
total_iteration = 0
def optimize(num_iteration):
  global total_iteration
  
  start_time = time.time()
  for i in range(total_iteration, total_iteration + num_iteration):
    #generate data in batch
    feed_dict_train = {x: x_batch,
                      y_true: y_ture_batch}
    
    session.run(optimizer, feed_dict=feed_dict_train)
    
    if i%100 == 0:
      acc = session.run(accuracy, feed_dict=feed_dict_train)
      print(acc)
  total_iteration += num_iteration
  end_time = time.time()
  time_diff = end_time - start_time
  
  print(time_diff)

In [0]:
# test_batch_size = 200
def print_test_accuracy(testx,testy):
  num_test = testx.shape[0]
  feed_dict = {x: testx,
               y_true: testy}
  
  